In [245]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import sqlite3
import plotly.graph_objects as go

In [277]:
import sqlalchemy as db

# Create databse. This commenad creates catmeals.sqlite automatically
engine = db.create_engine('sqlite:///catmeals.sqlite') 
connection = engine.connect()
metadata = db.MetaData()

# Create db model
cat_meals_dbTable = db.Table('emp', metadata,
                             db.Column('Id', db.Integer()),
                             db.Column('time_stamp', db.DateTime(), nullable=False),
                             db.Column('next_meals', db.DateTime(), nullable=False),
                             db.Column('remaining_meals', db.Integer()))
               
# Creates the table
metadata.create_all(engine)

In [279]:
# Open connection
con = sqlite3.connect("catmeals.sqlite")

# Write results 
a.to_sql("cat_meals", con, if_exists = "append")

# Close connection
con.close()

In [119]:
x = input("Catmate filled?: (Y|n) ")

Catmate filled?: (Y|n) Y


In [239]:
standard_date_format = "%Y-%m-%d %H:%M:%S"

def perdelta(start, end, delta):
    # generates a range of dates separated with start and end dates and a delta
    curr = start
    while curr < end:
        yield curr
        curr += delta

def nearest(items, pivot):
    # calculates the nearest item in a list from a pivot item
    return min(items, key=lambda x: abs(x - pivot))
        
""" Function to fill a table with future meal time stamps
Args: 
    time_stamp_now (datetime)

Returns:
    list: with date for next meals
"""
def generateNextMeals(time_stamp_now = datetime.now()): 
    today_evening = datetime.strptime(time_stamp_now.strftime("%Y-%m-%d") + " 19:00:00", standard_date_format)
    today_morning = datetime.strptime(time_stamp_now.strftime("%Y-%m-%d") + " 07:00:00", standard_date_format)

    # Assign next meal to next following date
    if(today_evening > time_stamp_now):
        next_meal = today_evening
    else:
        next_meal = today_morning
    
    # Generate following meals for the next days. 
    # Our catmate is programmed to rotate every 12 hours with 4 slots = 2 days
    date_list = []
    for date_item in perdelta(next_meal, next_meal+timedelta(days=2), timedelta(days=0.5)):
        date_list.append(date_item)
        
    return(date_list)

    
""" Function to retrieve how many meals are left
Args:
    meals_table (DataFrame)

Returns:
    remaining_meals (int) 
"""
def calculateRemainingMeals(meals_table):
    b = a[a.next_meals == nearest(a['next_meals'], datetime.now())][['remaining_meals']]
    return(b.remaining_meals)

In [269]:
if (x == "Y"):
    timeStamp = datetime.now()
    a = {'time_stamp':timeStamp.strftime(standard_date_format),
         'next_meals': generateNextMeals(timeStamp), 
         'remaining_meals': np.linspace(start=4, stop=1, num=4)}
    a = pd.DataFrame(a)
    print(a)
    
calculateRemainingMeals(a)

            time_stamp          next_meals  remaining_meals
0  2020-02-28 19:39:16 2020-02-28 07:00:00              4.0
1  2020-02-28 19:39:16 2020-02-28 19:00:00              3.0
2  2020-02-28 19:39:16 2020-02-29 07:00:00              2.0
3  2020-02-28 19:39:16 2020-02-29 19:00:00              1.0


1    3.0
Name: remaining_meals, dtype: float64

In [303]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a['next_meals'], 
                         y=a['remaining_meals'],
                         name="Remaining cat meals",
                         line_color='deepskyblue',
                         opacity=0.8,
                         marker = dict(size = 20,
                                      color=[4, 3, 2, 1],
                                       colorscale='Greens'),
                         mode='markers'
                        ))


fig.update_layout(xaxis_range=[datetime.now() - timedelta(days=1),
                               datetime.now() + timedelta(days=1)],
                 title_text="", 
                 xaxis_rangeslider_visible=True,
                  template = "plotly_dark")
fig.show()